**PROCESADO YOUTUBE**

**1. Preparación del entorno**

In [3]:
try:
  from google.colab import auth
  auth.authenticate_user()
  import pandas as pd
except:
  !pip install google-cloud-storage
  !pip install pandas

!pip install emoji google-api-python-client unidecode num2words

from google.cloud import storage
import emoji
import numpy as np
import random
import pandas as pd
import string
import regex as re
import nltk

import matplotlib.pyplot as plt

from googleapiclient.discovery import build
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.utils import resample
from num2words import num2words

nltk.download('stopwords')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=07b70ceee477f503009a6fcded5d3678eb309236bb62f43cf9b386d9b51558b0
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
def descargar_de_bucket_origen(nombre_bucket_origen, nombre_blob_origen, nombre_archivo_destino):
    cliente = storage.Client()
    bucket = cliente.bucket(nombre_bucket_origen)
    blob = bucket.blob(nombre_blob_origen)
    blob.download_to_filename(nombre_archivo_destino)
    print(f"Archivo {nombre_blob_origen} descargado como {nombre_archivo_destino}.")

nombre_bucket_origen = 'ds-edw-raw-d1655b14'
nombre_bucket_procesado = 'ds-edw-processed-d1655b14'
nombre_blob_origen = 'CSV/comentarios.csv'
ruta_descarga_local = 'comentarios.csv'
descargar_de_bucket_origen(nombre_bucket_origen, nombre_blob_origen, ruta_descarga_local)

input_file = pd.read_csv(ruta_descarga_local)

Archivo CSV/comentarios.csv descargado como comentarios.csv.


**2. Pipeline**

In [14]:
def preprocess_data(input_file, output_file):
    # Cargar datos desde el archivo CSV
    comments = pd.read_csv(input_file)

    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F3B5-\U0001F3EF"
                               u"\U0001F700-\U0001F77F"
                               u"\U0001F7E0-\U0001F7EB"
                               u"\U0001F0A0-\U0001F0FF"
                               u"\U0001F000-\U0001F02F"
                               u"\U0001F030-\U0001F09F"
                               u"\U0001F200-\U0001F2FF"
                               "]+", flags=re.UNICODE)

    def emoji_processed(df):
        def extract_and_count_emojis(text):
            if isinstance(text, str):
                return emoji_pattern.findall(text)
            return []

        emoji_list = []
        for index, row in df.iterrows():
            emojis = extract_and_count_emojis(row['Comentario'])
            if emojis:
                for emoji in emojis:
                    emoji_list.append({'emoji': emoji})

        all_emoji = pd.DataFrame(emoji_list, columns=['emoji'])

        # Calcular el porcentaje de aparición de cada emoji
        emoji_counts = all_emoji['emoji'].value_counts(normalize=True) * 100
        emoji_counts = emoji_counts.rename_axis('emoji').reset_index(name='% del total')
        emoji_counts['% del total'] = emoji_counts['% del total'].round(2)

        # Ordenar los emojis por porcentaje de mayor a menor
        emoji_counts.sort_values(by='% del total', ascending=False, inplace=True)

        # Guardar el DataFrame ordenado en un archivo CSV
        emoji_counts.to_csv("new_emoji.csv", index=False)

    def remove_emojis(text):
        if isinstance(text, str):
            return emoji_pattern.sub('', text)
        return text

    def clean_text(text):
        text = remove_emojis(text)
        if isinstance(text, str):
            text = re.sub(r'@[A-Za-z0-9_]+', '', text)
            text = re.sub(r'[^\w\s]', '', text)
            text = unidecode(text)
            text = text.lower()
            text = text.replace('!', '')
            words = [num2words(word, lang='es') if word.isdigit() else word for word in text.split()]
            text = ' '.join(words)
        return text

    # Procesar emojis y guardar en archivo CSV
    emoji_processed(comments)

    # Aplicar la función de limpieza de texto a la columna 'Comentario'
    comments['new_processed'] = comments['Comentario'].apply(clean_text)
    comments.drop('Comentario', axis=1, inplace=True)
    comments.replace('', np.nan, inplace=True)
    comments = comments.dropna()

    # Guardar el DataFrame procesado en un archivo CSV
    comments.to_csv(output_file, index=False)
    return comments

if __name__ == "__main__":
    input_file = 'comentarios.csv'
    output_file = 'new_processed.csv'

    processed_data = preprocess_data(input_file, output_file)

    print(f'Los datos preprocesados han sido guardados en: {output_file}')

    processed_data = pd.read_csv(output_file)
    processed_data.info()
    print("." * 20)

    for index, row in processed_data.head(5).iterrows():
        print("Autor:", row['Autor'])
        print("Comentario:", row['new_processed'])
        print()

    new_emoji = pd.read_csv('new_emoji.csv')
    print("." * 20)
    print(new_emoji.head(6))


def subir_a_bucket_destino(nombre_bucket_procesado, nombre_archivo_destino, nombre_blob_destino):
    cliente = storage.Client()
    bucket = cliente.bucket(nombre_bucket_procesado)
    blob = bucket.blob(nombre_blob_destino)
    # Verificar si el archivo ya existe en el bucket
    if blob.exists():
        print(f"El archivo {nombre_blob_destino} ya existe en el bucket {nombre_bucket_procesado} y no se ha subido por no sobreescribirlo y preservar los datos, pon otro nombre o solicita ayuda al administrador del bucket.")
    else:
        blob.upload_from_filename(nombre_archivo_destino)
        print(f"Archivo {nombre_archivo_destino} subido a {nombre_blob_destino} en el bucket {nombre_bucket_procesado}.")


nombre_bucket_procesado = 'ds-edw-processed-d1655b14'
nombre_blob_destino = 'CSV/new_processed.csv'
subir_a_bucket_destino(nombre_bucket_procesado, output_file, nombre_blob_destino)



Los datos preprocesados han sido guardados en: new_processed.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2001 entries, 0 to 2000
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Autor          2001 non-null   object
 1   new_processed  2001 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB
....................
Autor: @mipalacioazul4191
Comentario: el mismo tema que trata la pelicula la llave maestra me suena muy probable

Autor: @marinahernandezhernandez9814
Comentario: teneis razon en decir que todo es mentira incluso vuestros programas

Autor: @marinahernandezhernandez9814
Comentario: pero tu no tenias por tu vida por karles torah

Autor: @bellaguerra3183
Comentario: increible pero me lo imaginaba

Autor: @samadhi7453
Comentario: esse povo nao morre troca de pele

....................
  emoji  % del total
0     😮         7.96
1     😊         4.66
2     😂         4.66
3     🌎         2.72
4   